In [1]:
# import
import importlib
import pandas as pd
from pipeline_executor import PipelineExecutor
import nlp_analysis.word_wizard as ww

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/V

In [11]:
# get data
topic = "roche"

pipeline_executor = PipelineExecutor()
roche = pipeline_executor.execute(query=topic, max_articles=None, overwrite=True)

# safe and load quantum_df using parquet
roche.to_parquet('data/clean/roche.gzip', compression='gzip')
roche = pd.read_parquet('data/clean/roche.gzip')

Getting news article info: 100%|██████████| 588/588 [07:01<00:00,  1.39it/s]


In [28]:
roche = pd.read_parquet('data/clean/roche.gzip')
roche = roche.sample(n=20)
importlib.reload(ww)
wizard = ww.WordWizard(df=roche, interest = 'paragraph')
wizard.create_word_embeddings(wizard.interest, lean=True)
wizard.create_sentence_embeddings(wizard.interest)
wizard.cluster_embeddings(wizard.interest)
wizard.entitiy_recognition(wizard.interest)
wizard.summarize_medoids(wizard.interest)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Extracting organizations for column paragraph: 100%|██████████| 4/4 [00:01<00:00,  3.99it/s]
Creating summaries for medoids of column paragraph: 100%|███████

In [25]:
# silhouette score ain't working with too few samples I think
from sklearn.cluster import KMeans
df = wizard.df
from sklearn.metrics import silhouette_score
column = 'paragraph'
k=2
embed_column = column + wizard.EMB_SUFFIX
kk = KMeans(n_clusters=k, n_init='auto').fit(df[embed_column].tolist())
labels = kk.labels_
silhouette_score(df[embed_column].tolist(), labels, metric='euclidean')

0.18544431

In [7]:
# check the number of entities in each cluster
wizard.df['body_word_embeddings_clusters'].value_counts()

1    14
0     8
2     7
Name: body_word_embeddings_clusters, dtype: int64

In [8]:
# check the number of medoids per cluster
wizard.df[wizard.df['body_word_embeddings_clusters_medoids'] == True][['body_word_embeddings', 'body_word_embeddings_clusters']]

,body_word_embeddings,body_word_embeddings_clusters
0,"[0.06478899, -0.13545708, 0.06179996, 0.215222...",1
11,"[0.109405935, -0.30240607, -0.03303288, 0.2109...",0
13,"[0.09361677, -0.030796781, -0.11409727, 0.0525...",2
19,"[0.15661505, -0.10956453, -0.25051394, 0.18294...",0
21,"[0.13517544, 0.0073455237, -0.15610743, 0.0545...",2
28,"[0.10058438, -0.09042827, -0.029156672, 0.0413...",1
